In [1]:
%matplotlib inline

import pandas as pd
import numpy as np

from sklearn.linear_model import  LinearRegression
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor


from statsmodels.tsa.statespace.sarimax import SARIMAXResults
from numpy.linalg.linalg import LinAlgError

from scipy import stats

from matplotlib import pyplot as plt
from statsmodels import api as sm
from tqdm import tqdm 
import warnings
from itertools import product

Подготовим список 102 отобранных на 2 неделе районов в мае

In [2]:
#Загружаем предварительно подготовленную в 1-м задании таблицу тайм сессии для мая 2016
df=pd.read_csv('TS_2016-05.csv',header=-1)
cols=[]
for i in range(1,2501):
    x=df[i].mean()
    if x>=5:
        cols.append(i)
print u'Число отобранных районов', len(cols)

Число отобранных районов 102


In [3]:
df_05=pd.read_csv('TS_2016-05.csv',header=-1,parse_dates=[0],index_col=[0])[cols]
df_06=pd.read_csv('TS_2016-06.csv',header=-1,parse_dates=[0],index_col=[0])[cols]
files=['TS_2015-11.csv','TS_2015-12.csv','TS_2016-01.csv','TS_2016-02.csv','TS_2016-03.csv','TS_2016-04.csv']
df_tr=pd.DataFrame()
for i,f in enumerate(files):
    df_first=pd.read_csv(f,header=-1,parse_dates=[0],index_col=[0])
    df_tr=df_tr.append (df_first,ignore_index=False)
df_tr=df_tr[cols]
df_tr.shape

(4368, 102)

In [4]:
df_all=pd.concat((df_tr, df_05,df_06))
df_all.shape

(5832, 102)

#### 1. Для каждой из шести задач прогнозирования y^T+i|T,i=1,…,6 сформируйте выборки. Откликом будет yT+i при всевозможных значениях T, а признаки можно использовать следующие:


In [5]:
#Функция формирует временные признаки (не зависящие от географической зоны)
def get_time_features(df, K):
    length = df.shape[0] + 1
    features = pd.DataFrame()
    for i in range(1,K+1):
        features['sinw_w%d' % i] = [ np.sin(t * i * 2.*np.pi / 168.) for t in np.arange(1, length)]
        features['cosw_w%d' % i] = [ np.cos(t * i * 2.*np.pi / 168.) for t in np.arange(1, length)]
        features['sind_%d' % i] = [ np.sin(t * i * 2.*np.pi / 24.) for t in np.arange(1, length)]
        features['cosd_%d' % i] = [ np.cos(t * i * 2.*np.pi / 24.) for t in np.arange(1, length)]

    features[['h%d' % i for i in range(0, 24)]] = pd.get_dummies(df.index.hour)
    features[['dw%d' % i for i in range(0, 7)]] = pd.get_dummies(df.index.weekday)
    features[['dm%d' % i for i in range(0, 31)]] = pd.get_dummies(df.index.day)
    features['is_weekend'] = (df.index.weekday > 4).astype(float)
      
    return features

In [6]:
#Кумулятивное скользящее
def mov_cum (z,N=3):
    cumsum, moving_aves = [0], []
    for i, x in enumerate(z, 1):
        cumsum.append(cumsum[i-1] + x)
        if i>=N:
            moving_ave = (cumsum[i] - cumsum[i-N])
        
            moving_aves.append(moving_ave)
            
    return [0]*(N-1)+moving_aves    

In [7]:
#Зональные регрессионные признаки
def get_trip_features(df,Region,K):
    L=df.shape[0]
    S=df[Region].values
    features = pd.DataFrame()
    y0=np.zeros(K)
    
    #количество поездок из рассматриваемого района в моменты времени yT,yT−1,…,yT−K 
    for j in range(K):
        y=[S[i-j] for i in range(K,L)]
        
        features['n-_%d' % j]=np.concatenate((y0,y),axis=0)
                 
    #суммарное количество поездок из рассматриваемого района за предшествующие полдня, сутки, неделю, месяц                 
    hours=[4,8,12,24,24*7,24*30]  
    for h in hours:
        features['sum_%d' % h]=mov_cum(S,N=h)
    
    return  features        

In [8]:
df_ARIMA=pd.read_csv('ARIMA_pred(15.11-16.06)',',') #предсказания по модели ARIMA

Приготовим X регрессионных признаков и 6 ответов y

In [9]:
#Формирует df - регрессионные данные Х и ответы y - 6 вариантов ответов для шага прогноза (1,...6)
def get_Xy_data(df, Kt = 20, Kd=10):
    Xtime=get_time_features(df, Kt)

    data=[]
    for j,r in enumerate(cols): #перебор по зонам
        #if j==1:
        #    break
        Xtrip=get_trip_features(df, r, Kd)
        X=pd.concat((Xtime,Xtrip), axis=1) # объединение временных признаков и зональных
        
        x_arima=df_ARIMA[str(r)]
        y=df_all[r]
        
        # добавление столбцов предсказаний ARIMA
        for i in range(1,7): 
            X['ARIMA_%d' % i]=x_arima.shift(-i).values
        
        if j==0:
            X_size=X.shape[1] # ширина таблицы регрессионных признаков
        
        # добавление столбцов ответов
        for i in range(1,7): 
            X['y_%d' % i]=y.shift(-i).values
        
        X.index=df.index
        
        #Обрезка 0 в начале и 6 строк в конце
        X=X.iloc[24*30-1:df.shape[0]-6] 
        data.append(X)
    return data, X_size

In [10]:
%%time
data, X_size =get_Xy_data(df_all, Kt = 20, Kd=10)
print X_size

165
Wall time: 13.7 s


In [11]:
data[0].head()

,sinw_w1,cosw_w1,sind_1,cosd_1,sinw_w2,cosw_w2,sind_2,cosd_2,sinw_w3,cosw_w3,...,ARIMA_3,ARIMA_4,ARIMA_5,ARIMA_6,y_1,y_2,y_3,y_4,y_5,y_6
0,,,,,,,,,,,,,,,,,,,,,
2015-11-30 23:00:00,0.974928,-0.222521,6.862974e-15,1.000000,-0.433884,-0.900969,1.372595e-14,1.000000e+00,-0.781831,0.623490,...,10.444599,3.255400,3.330129,3.999257,19.0,16.0,8.0,3.0,2.0,7.0
2015-12-01 00:00:00,0.965926,-0.258819,2.588190e-01,0.965926,-0.500000,-0.866025,5.000000e-01,8.660254e-01,-0.707107,0.707107,...,3.255400,3.330129,3.999257,12.881411,16.0,8.0,3.0,2.0,7.0,20.0
2015-12-01 01:00:00,0.955573,-0.294755,5.000000e-01,0.866025,-0.563320,-0.826239,8.660254e-01,5.000000e-01,-0.623490,0.781831,...,3.330129,3.999257,12.881411,24.737939,8.0,3.0,2.0,7.0,20.0,54.0
2015-12-01 02:00:00,0.943883,-0.330279,7.071068e-01,0.707107,-0.623490,-0.781831,1.000000e+00,-8.335645e-15,-0.532032,0.846724,...,3.999257,12.881411,24.737939,55.497679,3.0,2.0,7.0,20.0,54.0,94.0
2015-12-01 03:00:00,0.930874,-0.365341,8.660254e-01,0.500000,-0.680173,-0.733052,8.660254e-01,-5.000000e-01,-0.433884,0.900969,...,12.881411,24.737939,55.497679,85.346650,2.0,7.0,20.0,54.0,94.0,81.0


#### 3. Выберите вашу любимую регрессионную модель и настройте её на каждом из шести наборов данных, подбирая гиперпараметры на мае 2016. Желательно, чтобы модель:

допускала попарные взаимодействия между признаками
была устойчивой к избыточному количеству признаков (например, использовала регуляризаторы)
#### 4. Выбранными моделями постройте для каждой географической зоны и каждого конца истории от 2016.04.30 23:00 до 2016.05.31 17:00 прогнозы на 6 часов вперёд; посчитайте в ноутбуке ошибку прогноза по следующему функционалу:

In [12]:
#Составим предсказательные модели для каждой зоны, каждаго предсказательного шага смещения

#Линейная регрессия

def get_LinearRegr_models(time_end='2016-04-30 17:00:00'):
    '2016-04-30 17:00:00' #Данные для создания моделей обрезаются этим концом истории
    models=[]
    for dat in data:
        X=dat[:time_end].iloc[:,:X_size]
        m=[]
        for j in range(6):
            y=dat[:time_end].iloc[:,X_size+j]
            m.append(LinearRegression().fit(X, y))
        models.append(m)
    return models

In [13]:
#Лассо регрессия
def get_Lasso_models(time_end='2016-04-30 17:00:00',alpha=0.1):
    '2016-04-30 17:00:00' #Данные для создания моделей обрезаются этим концом истории
    warnings.filterwarnings('ignore')
    models=[]
    for dat in data:
        X=dat[:time_end].iloc[:,:X_size]
        m=[]
        for j in range(6):
            y=dat[:time_end].iloc[:,X_size+j]
            m.append(Lasso(alpha).fit(X, y))
        models.append(m)
    warnings.filterwarnings('default')
    return models

In [14]:
#Случайный лес
def get_rf_models(time_end='2016-04-30 17:00:00'):
    '2016-04-30 17:00:00' #Данные для создания моделей обрезаются этим концом истории
    warnings.filterwarnings('ignore')
    models=[]
    for dat in data:
        X=dat[:time_end].iloc[:,:X_size]
        m=[]
        for j in range(6):
            y=dat[:time_end].iloc[:,X_size+j]
            m.append(RandomForestRegressor().fit(X, y))
        models.append(m)
    warnings.filterwarnings('default')
    return models

In [15]:
%%time
LR_models=get_LinearRegr_models(time_end='2016-04-30 17:00:00') #Получение списка моделей

Wall time: 28.7 s


In [16]:
%%time
Lasso_models=get_Lasso_models(time_end='2016-04-30 17:00:00',alpha=0.05) #Получение списка моделей

Wall time: 5min 54s


In [17]:
%%time
RF_models=get_rf_models(time_end='2016-04-30 17:00:00') #Получение списка моделей

Wall time: 15min 55s


In [18]:
time_ends=df_all['2016-04-30 23:00:00':'2016-05-31 17:00:00'].index

In [19]:
def Qmay(models):
    q=0
    for r in range(len(cols)): #преребор по районам
        dat=data[r]
        model=models[r]
        if r==10:
            break
   
        for i,t in enumerate(time_ends): #перебор по концам истории
            X=dat[t:t].iloc[:,:X_size]
        
            for j in range(6): #перебор по шагам прогноза
            
                m=model[j]
                y_hat=m.predict(X)
                y_real=dat[t:t].iloc[:,X_size+j]
                #print y_hat[0],y_real[0]
                q+=np.abs(y_hat[0]-y_real[0])
            
    Qmay=1./(739*6)*q
    return Qmay

In [20]:
%%time
print Qmay(LR_models)
print Qmay(Lasso_models)
print Qmay(RF_models)

41.60485644941389
45.93634335357664
91.5516463689671
Wall time: 3min 49s


#### Вывод: из 3 видов моделей самый лучший результат на функционале Qmay дает Линейная регрессия, на втором месте Лассо, на третьем - случайный лес.
#### Ошибка уменьшилась в 2 раза. Сама по себе ARIMA давала 83 

#### 5. Итоговыми моделями постройте прогнозы для каждого конца истории от 2016.05.31 23:00 до 2016.06.30 17:00 и запишите все результаты в один файл в формате geoID, histEndDay, histEndHour, step, y. Здесь geoID — идентификатор зоны, histEndDay — день конца истории в формате id,y, где столбец id состоит из склеенных через подчёркивание идентификатора географической зоны, даты конца истории, часа конца истории и номера отсчёта, на который делается предсказание (1-6); столбец y — ваш прогноз.

In [21]:
#Далее будем использовать Линейную модель
models=LR_models

In [22]:
time_ends=df_all['2016-05-31 23:00:00':'2016-06-30 17:00:00'].index


In [23]:
%%time
Q=[]
for r in range(len(cols)): #преребор по районам
    dat=data[r]
    model=models[r]
    i_zone=cols[r]
    #if r==1:
        #break
        
    for i,t in enumerate(time_ends): #перебор по концам истории
        X=dat[t:t].iloc[:,:X_size]
        
        for j in range(6): #перебор по шагам прогноза
            
            m=model[j]
            y_hat=m.predict(X)[0]
            
            te=time_ends[i]
            id_='{}_2016-{:02}-{:02}_{}_{}'.format(i_zone, te.month, te.day, te.hour, j+1)
            
            Q.append([id_, y_hat])

Predict_june=pd.DataFrame(Q)
Predict_june.columns=['id','y']

Wall time: 6min 18s


In [24]:
Predict_june.to_csv('Kaggle2.csv',',',index=False)

In [25]:
Predict_june.head()

,id,y
0,1075_2016-05-31_23_1,27.794346
1,1075_2016-05-31_23_2,26.320137
2,1075_2016-05-31_23_3,25.833581
3,1075_2016-05-31_23_4,25.065827
4,1075_2016-05-31_23_5,26.700609


#### 6. Загрузите полученный файл на kaggle: https://inclass.kaggle.com/c/yellowtaxi. Добавьте в ноутбук ссылку на сабмишн.

Ссылка на сабмишн
https://inclass.kaggle.com/c/yellowtaxi/submit
Score 52.20717   